In [ ]:
import os
import datetime
import logging
from typing import Any, Dict, List

from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

from google.adk.agents import Agent, SequentialAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import FunctionTool
from google.adk.tools.tool_context import ToolContext
from google.genai import types

# ===================== CONFIG =====================

# Your Gemini API key here (you can also move this to .env)
os.environ["GOOGLE_API_KEY"] = "Enter your google api key"

APP_NAME = "jobbuddy"
USER_ID = "demo_user"
SESSION_ID = "demo_session"

MODEL_ID = "gemini-2.5-flash"

# Gmail (read-only) + Sheets (read/write)
SCOPES = [
    "https://www.googleapis.com/auth/gmail.readonly",
    "https://www.googleapis.com/auth/spreadsheets",
]

# Your Sheet config
JOBBUDDY_SHEET_ID = "Enter your sheet id"
JOBBUDDY_SHEET_NAME = "Applications"  # must match tab name

TRACKER_COLUMNS = [
    "application_id",
    "company",
    "role",
    "source",
    "status",
    "applied_date",
    "last_activity_date",
    "next_action",
    "priority",
    "thread_id",
]

# Optional retry config for Gemini
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s] %(levelname)s %(name)s - %(message)s",
)
logger = logging.getLogger("jobbuddy")


# ===================== OAuth helper =====================

def get_creds() -> Credentials:
    """
    OAuth helper for local VS Code / desktop using a Desktop OAuth client.

    Flow:
    1. If token.json exists & is valid -> reuse it.
    2. Else:
       - Use InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
       - Run a local server for the OAuth callback.
    """
    creds: Credentials | None = None

    # Reuse token if we have one
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)

    # If no valid creds, start the flow
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            if not os.path.exists("credentials.json"):
                raise FileNotFoundError(
                    "credentials.json not found. Place your Desktop OAuth client JSON "
                    "in the project root (same folder where you run this script)."
                )

            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json",
                scopes=SCOPES,
            )

            creds = flow.run_local_server(
                port=0,
                open_browser=True,
                prompt="consent",
            )

        # Save token for future runs
        with open("token.json", "w") as token_file:
            token_file.write(creds.to_json())

    return creds


creds = get_creds()  # <-- will open browser on first run

gmail_service = build("gmail", "v1", credentials=creds)
sheets_service = build("sheets", "v4", credentials=creds)

print("✅ Gmail + Sheets clients ready")


# ===================== TOOL 1: Gmail fetch =====================

def fetch_recent_job_emails(max_results: int, tool_context: ToolContext) -> Dict[str, Any]:
    """
    Use Gmail API to fetch recent job application–style emails.

    Writes:
      tool_context.state["raw_job_emails"] = [ {thread_id, subject, from, snippet, received_at, ...}, ... ]
    Returns the same list in the tool result so the agent can see it.
    """
    logger.info(f"[Tool] fetch_recent_job_emails called with max_results={max_results}")

    # Focus on actual applications / confirmations, not alerts / newsletters
    query = (
        '(from:jobs-noreply@linkedin.com '
        'OR subject:"Your application" '
        'OR subject:"Application received" '
        'OR subject:"Thank you for applying" '
        'OR subject:"was sent" '
        'OR subject:"status" '
        'OR subject:"update" '
        'OR subject:"applied to")'
    )
    print("DEBUG query in tool:", query)

    response = gmail_service.users().messages().list(
        userId="me",
        q=query,
        maxResults=max_results,
    ).execute()

    size = response.get("resultSizeEstimate")
    message_refs = response.get("messages", [])
    print("DEBUG resultSizeEstimate:", size, "len(messages):", len(message_refs))

    emails: List[Dict[str, Any]] = []

    for ref in message_refs or []:
        msg = gmail_service.users().messages().get(
            userId="me",
            id=ref["id"],
            format="metadata",
            metadataHeaders=["From", "Subject", "Date", "List-Id", "X-Source"],
        ).execute()

        headers = {
            h["name"].lower(): h["value"]
            for h in msg.get("payload", {}).get("headers", [])
        }
        snippet = msg.get("snippet", "")
        internal_ts = int(msg.get("internalDate", 0)) / 1000.0
        received_at = datetime.datetime.utcfromtimestamp(internal_ts).isoformat() + "Z"

        emails.append(
            {
                "thread_id": msg.get("threadId"),
                "message_id": msg.get("id"),
                "from": headers.get("from", ""),
                "subject": headers.get("subject", ""),
                "snippet": snippet,
                "received_at": received_at,
                "raw_headers": {
                    "date": headers.get("date"),
                    "list-id": headers.get("list-id"),
                    "x-source": headers.get("x-source"),
                },
            }
        )

    tool_context.state["raw_job_emails"] = emails

    return {
        "status": "success",
        "count": len(emails),
        "emails": emails,
    }


fetch_emails_tool = FunctionTool(func=fetch_recent_job_emails)


# ===================== TOOL 2: Save parsed applications (LLM → state) =====================

def save_parsed_applications(
    parsed_applications: List[Dict[str, Any]],
    tool_context: ToolContext,
) -> Dict[str, Any]:
    """
    Save the LLM-parsed applications into tool_context.state['parsed_applications'].

    The agent should pass a list like:
    [
      {
        "thread_id": "...",
        "company": "...",
        "role": "...",
        "source": "LinkedIn" | "Company ATS" | "Company Site" | "Unknown",
        "status": "Applied" | "Screening" | "Interview" | "Rejected" | "Offer",
        "applied_date": "YYYY-MM-DD",
        "last_email_date": "YYYY-MM-DDTHH:MM:SSZ",
        "interview_date": "YYYY-MM-DD" | null
      },
      ...
    ]
    """
    logger.info("[Tool] save_parsed_applications called")
    tool_context.state["parsed_applications"] = parsed_applications or []
    return {
        "status": "success",
        "count": len(parsed_applications or []),
    }


save_parsed_apps_tool = FunctionTool(func=save_parsed_applications)


# ===================== TOOL 3: Load tracker snapshot from Sheet =====================

def load_tracker_snapshot(tool_context: ToolContext) -> Dict[str, Any]:
    """
    Read JobBuddy tracker rows from the Google Sheet into state["tracker_rows"].
    """
    logger.info("[Tool] load_tracker_snapshot called")

    if JOBBUDDY_SHEET_ID == "JOBBUDDY_SHEET_ID":
        raise ValueError("Set JOBBUDDY_SHEET_ID to your real Sheet ID.")

    range_ = f"{JOBBUDDY_SHEET_NAME}!A:Z"
    result = sheets_service.spreadsheets().values().get(
        spreadsheetId=JOBBUDDY_SHEET_ID,
        range=range_,
    ).execute()

    values = result.get("values", [])
    if not values:
        tool_context.state["tracker_rows"] = []
        return {"status": "success", "row_count": 0}

    header = values[0]
    data_rows = values[1:]

    if header[: len(TRACKER_COLUMNS)] != TRACKER_COLUMNS:
        raise ValueError(
            f"Sheet header mismatch.\nExpected: {TRACKER_COLUMNS}\nGot: {header}"
        )

    tracker_rows: List[Dict[str, Any]] = []
    for i, row in enumerate(data_rows, start=2):  # Sheet row index
        row_padded = row + [""] * (len(TRACKER_COLUMNS) - len(row))
        row_dict = {
            col: row_padded[idx]
            for idx, col in enumerate(TRACKER_COLUMNS)
        }
        row_dict["sheet_row"] = i
        tracker_rows.append(row_dict)

    tool_context.state["tracker_rows"] = tracker_rows

    logger.info(f"[Tool] Loaded {len(tracker_rows)} tracker rows from sheet.")
    return {"status": "success", "row_count": len(tracker_rows)}


load_tracker_tool = FunctionTool(func=load_tracker_snapshot)


# ===================== TOOL 4: Sync tracker to Sheet =====================

def _row_dict_to_sheet_values(row: Dict[str, Any]) -> List[Any]:
    return [row.get(col, "") for col in TRACKER_COLUMNS]


def sync_tracker_sheet(tool_context: ToolContext) -> Dict[str, Any]:
    """
    Idempotently upsert parsed applications into tracker_rows and sync to the sheet.
    """
    logger.info("[Tool] sync_tracker_sheet called")

    state = tool_context.state
    parsed_apps: List[Dict[str, Any]] = state.get("parsed_applications", [])
    tracker_rows: List[Dict[str, Any]] = state.get("tracker_rows", [])

    index_by_thread = {
        row["thread_id"]: row for row in tracker_rows if row.get("thread_id")
    }

    new_rows_to_append: List[Dict[str, Any]] = []
    changed_existing_rows: List[Dict[str, Any]] = []

    new_rows = 0
    updated_rows = 0

    for app in parsed_apps:
        thread_id = app.get("thread_id")
        if not thread_id:
            continue

        existing = index_by_thread.get(thread_id)

        if existing is None:
            new_row = {
                "application_id": app.get("application_id") or f"app_{len(tracker_rows) + len(new_rows_to_append) + 1}",
                "company": app.get("company"),
                "role": app.get("role"),
                "source": app.get("source"),
                "status": app.get("status"),
                "applied_date": app.get("applied_date"),
                "last_activity_date": (
                    app.get("last_email_date", "")[:10] if app.get("last_email_date") else ""
                ),
                "next_action": "",
                "priority": "",
                "thread_id": thread_id,
                "sheet_row": None,
            }
            new_rows_to_append.append(new_row)
            index_by_thread[thread_id] = new_row
            new_rows += 1

        else:
            changed = False

            # Refresh all logical fields when we re-parse the same thread
            for key in ["company", "role", "source", "status", "applied_date"]:
                if app.get(key) and existing.get(key) != app.get(key):
                    existing[key] = app[key]
                    changed = True

            # last_activity_date from latest email
            if app.get("last_email_date"):
                led = app["last_email_date"][:10]
                if existing.get("last_activity_date") != led:
                    existing["last_activity_date"] = led
                    changed = True

            if changed:
                changed_existing_rows.append(existing)
                updated_rows += 1

    # Update in-memory tracker
    tracker_rows.extend(new_rows_to_append)
    state["tracker_rows"] = tracker_rows

    # Append new rows to Sheet
    if new_rows_to_append:
        values_to_append = [_row_dict_to_sheet_values(r) for r in new_rows_to_append]
        body = {"values": values_to_append}
        sheets_service.spreadsheets().values().append(
            spreadsheetId=JOBBUDDY_SHEET_ID,
            range=f"{JOBBUDDY_SHEET_NAME}!A2",
            valueInputOption="USER_ENTERED",
            insertDataOption="INSERT_ROWS",
            body=body,
        ).execute()
        logger.info(f"[Tool] Appended {len(values_to_append)} new rows to sheet.")

    # Batch update existing rows
    if changed_existing_rows:
        data = []
        for row in changed_existing_rows:
            sr = row.get("sheet_row")
            if not sr:
                continue
            data.append(
                {
                    "range": f"{JOBBUDDY_SHEET_NAME}!A{sr}:J{sr}",
                    "values": [_row_dict_to_sheet_values(row)],
                }
            )
        if data:
            body = {
                "valueInputOption": "USER_ENTERED",
                "data": data,
            }
            sheets_service.spreadsheets().values().batchUpdate(
                spreadsheetId=JOBBUDDY_SHEET_ID,
                body=body,
            ).execute()
            logger.info(f"[Tool] Updated {len(data)} existing rows in sheet.")

    total_rows = len(tracker_rows)
    logger.info(
        f"[Tool] sync_tracker_sheet done: new_rows={new_rows}, updated_rows={updated_rows}, total={total_rows}"
    )

    return {
        "status": "success",
        "new_rows": new_rows,
        "updated_rows": updated_rows,
        "total_rows": total_rows,
    }


sync_tracker_tool = FunctionTool(func=sync_tracker_sheet)


# ===================== TOOL 5: Compute insights =====================

def compute_insights(tool_context: ToolContext) -> Dict[str, Any]:
    """
    Compute weekly stats + stalled applications from tracker_rows.
    """
    logger.info("[Tool] compute_insights called")

    state = tool_context.state
    tracker_rows: List[Dict[str, Any]] = state.get("tracker_rows", [])

    today = datetime.date.today()
    seven_days_ago = today - datetime.timedelta(days=7)
    three_days_ago = today - datetime.timedelta(days=3)

    applied_this_week = 0
    moved_to_interview = 0
    rejected_this_week = 0
    stalled: List[Dict[str, Any]] = []

    for row in tracker_rows:
        applied_str = row.get("applied_date")
        last_str = row.get("last_activity_date")
        status = (row.get("status") or "").lower()

        applied_date = datetime.date.fromisoformat(applied_str) if applied_str else None
        last_activity = datetime.date.fromisoformat(last_str) if last_str else None

        if applied_date and applied_date >= seven_days_ago:
            applied_this_week += 1
        if status == "interview" and last_activity and last_activity >= seven_days_ago:
            moved_to_interview += 1
        if status == "rejected" and last_activity and last_activity >= seven_days_ago:
            rejected_this_week += 1

        if status in {"applied", "screening", "interview"} and last_activity:
            if last_activity <= three_days_ago:
                stalled.append(
                    {
                        "company": row.get("company"),
                        "role": row.get("role"),
                        "status": row.get("status"),
                        "last_activity_date": row.get("last_activity_date"),
                        "thread_id": row.get("thread_id"),
                        "suggested_next_action": (
                            "Send a polite follow-up email to the recruiter."
                            if status in {"applied", "screening"}
                            else "Send a thank-you / check in on next steps."
                        ),
                    }
                )

    stats = {
        "applied_this_week": applied_this_week,
        "moved_to_interview": moved_to_interview,
        "rejected_this_week": rejected_this_week,
        "total_tracked": len(tracker_rows),
    }

    summary_lines = [
        f"You have {stats['total_tracked']} total applications.",
        f"This week: {applied_this_week} applied, {moved_to_interview} moved to interviews, "
        f"{rejected_this_week} rejected.",
        f"{len(stalled)} applications look stalled (no activity for 3+ days).",
    ]
    summary = " ".join(summary_lines)

    insights = {
        "summary": summary,
        "stats": stats,
        "stalled": stalled,
        "generated_at": today.isoformat(),
    }

    state["weekly_insights"] = insights

    return {
        "status": "success",
        **insights,
    }


compute_insights_tool = FunctionTool(func=compute_insights)


# ===================== InboxAgent (LLM does parsing) =====================

inbox_agent = Agent(
    model=MODEL_ID,
    name="InboxAgent",
    description="Parses Gmail job-related emails into normalized job applications.",
    instruction="""
You are the InboxAgent for JobBuddy.

Your job is to:
1. Call the tool `fetch_recent_job_emails(max_results)` to get recent job-related emails.
   - The tool returns a JSON object with a list `emails`.
   - Each email has at least: thread_id, from, subject, snippet, received_at, raw_headers.

2. From this tool output, decide for each email whether it is:
   - a new job application,
   - an update to an existing application (same thread_id),
   - or not job-related (ignore it).

3. Build a list `parsed_applications`, where each item is a JSON object:
   {
     "thread_id": str,
     "company": str,
     "role": str,
     "source": str | null,     // e.g. "LinkedIn", "Indeed", "Glassdoor", "Company ATS", "Company Site", "Unknown"
     "status": str,            // one of: "Applied", "Screening", "Interview", "Rejected", "Offer"
     "applied_date": str,      // YYYY-MM-DD (if unknown, infer a reasonable date from received_at)
     "last_email_date": str,   // ISO timestamp from the email's received_at
     "interview_date": str | null
   }

4. Then call the tool `save_parsed_applications(parsed_applications=...)`
   so that these results are written into session.state['parsed_applications'].

Rules:
- Be conservative: only mark an email as a job application if the subject/snippet clearly indicates
  an application confirmation, application received, or similar.
- Try your best to extract clear company and role names from the subject and snippet.
- Use the email "from" domain to guess the source when possible (e.g., linkedin.com -> "LinkedIn").
- If unsure about any field, set it to null (or a sensible default like "Unknown").
- Do NOT invent applications that aren't actually in the tool output.
- After using the tools, in your final answer, briefly summarize how many applications you found
  and their statuses.
""",
    tools=[fetch_emails_tool, save_parsed_apps_tool],
    output_key="inbox_summary",
)


# ===================== TrackerAgent =====================

tracker_agent = Agent(
    model=MODEL_ID,
    name="TrackerAgent",
    description="Keeps the JobBuddy Google Sheet tracker in sync with applications.",
    instruction="""
You are the TrackerAgent for JobBuddy.

1. Call `load_tracker_snapshot()` to load tracker rows into session.state['tracker_rows'].
2. Read `parsed_applications` from session.state (these were stored by InboxAgent via the tool).
3. Plan how to apply these to the tracker:
   - If thread_id already exists -> update that row.
   - Otherwise -> create a new row.
4. Call `sync_tracker_sheet()` to perform the idempotent upsert.

Do NOT manually mutate tracker_rows in your textual response; let the tool update the state.
In your final answer, report how many rows were created, updated, and the total rows.
""",
    tools=[load_tracker_tool, sync_tracker_tool],
    output_key="tracker_summary",
)


# ===================== InsightsAgent =====================

insights_agent = Agent(
    model=MODEL_ID,
    name="InsightsAgent",
    description="Summarizes weekly job search progress and reminders.",
    instruction="""
You are the InsightsAgent for JobBuddy.

1. Assume the tracker has already been synced and session.state['tracker_rows']
   is up to date.
2. Call `compute_insights()` to compute weekly stats and stalled applications.
3. Then, produce a concise, friendly weekly summary including:
   - How many roles were applied to this week.
   - How many moved to interviews.
   - How many were rejected.
   - A bulleted or clearly separated list of stalled applications and suggested follow-ups.

Tone: practical, encouraging, and specific.
""",
    tools=[compute_insights_tool],
    output_key="insights_summary",
)


# ===================== Multi-agent pipeline =====================

jobbuddy_pipeline = SequentialAgent(
    name="JobBuddyPipeline",
    sub_agents=[inbox_agent, tracker_agent, insights_agent],
)

session_service = InMemorySessionService()


session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
)

runner = Runner(
    agent=jobbuddy_pipeline,
    app_name=APP_NAME,
    session_service=session_service,
)

from google.genai import types as genai_types

def run_jobbuddy_once(user_message: str) -> str:
    """
    Run the JobBuddy pipeline once and return the final text from InsightsAgent.
    Assumes the session was already created with `await session_service.create_session(...)`.
    """
    content = genai_types.Content(
        role="user",
        parts=[genai_types.Part(text=user_message)],
    )

    final_text = "No response."

    events = runner.run(
        user_id=USER_ID,
        session_id=SESSION_ID,
        new_message=content,
    )

    for event in events:
        if event.is_final_response() and event.content and event.content.parts:
            final_text = event.content.parts[0].text

    return final_text


summary = run_jobbuddy_once(
    "Sync my job search from Gmail and tell me how my week went."
)
print(summary)


[2025-12-01 03:38:46,952] INFO googleapiclient.discovery_cache - file_cache is only supported with oauth2client<4.0.0
[2025-12-01 03:38:46,962] INFO googleapiclient.discovery_cache - file_cache is only supported with oauth2client<4.0.0
[2025-12-01 03:38:46,966] WARNING google_adk.google.adk.runners - App name mismatch detected. The runner is configured with app name "jobbuddy", but the root agent was loaded from "C:\Users\chait\OneDrive\Desktop\job-buddy\.venv\Lib\site-packages\google\adk\agents", which implies app name "agents".


✅ Gmail + Sheets clients ready


[2025-12-01 03:38:47,545] INFO google_adk.google.adk.models.google_llm - Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False
[2025-12-01 03:38:48,832] INFO httpx - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
[2025-12-01 03:38:48,842] INFO google_adk.google.adk.models.google_llm - Response received from the model.
[2025-12-01 03:38:48,842] WARNING google_genai.types - Warning: there are non-text parts in the response: ['function_call'], returning concatenated text result from text parts. Check the full candidates.content.parts accessor to get the full model response.
[2025-12-01 03:38:48,849] INFO jobbuddy - [Tool] fetch_recent_job_emails called with max_results=10


DEBUG query in tool: (from:jobs-noreply@linkedin.com OR subject:"Your application" OR subject:"Application received" OR subject:"Thank you for applying" OR subject:"was sent" OR subject:"status" OR subject:"update" OR subject:"applied to")
DEBUG resultSizeEstimate: 201 len(messages): 10


C:\Users\chait\AppData\Local\Temp\ipykernel_14348\1598437832.py:173: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  received_at = datetime.datetime.utcfromtimestamp(internal_ts).isoformat() + "Z"
[2025-12-01 03:38:52,317] INFO google_adk.google.adk.models.google_llm - Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False
[2025-12-01 03:39:04,195] INFO httpx - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
[2025-12-01 03:39:04,204] INFO google_adk.google.adk.models.google_llm - Response received from the model.
[2025-12-01 03:39:04,204] INFO jobbuddy - [Tool] save_parsed_applications called
[2025-12-01 03:39:04,780] INFO google_adk.google.adk.models.google_llm - Sending out reques

Here's a summary of your job search activity this week:

You applied to **14 roles** this week.
**1 application moved to an interview** stage.
**1 application was rejected**.

Here are some applications that look stalled and could use a follow-up:

*   **Reyes Beverage Group Corporate** (IT Applications Security Data Analyst): Send a polite follow-up email to the recruiter.
*   **Internet Brands** (Associate Data Analyst): Send a polite follow-up email to the recruiter.
*   **Snyk** (Strategic Analytics Associate): Send a polite follow-up email to the recruiter.
*   **Qemailserver** (Unknown role): Send a polite follow-up email to the recruiter.
*   **Danson Solutions** (Unknown): Send a thank-you / check in on next steps.
*   **Sony** (Associate Data Scientist): Send a polite follow-up email to the recruiter.
*   **Bimbo Bakeries USA** (Analytics Analyst): Send a polite follow-up email to the recruiter.
*   **Plymouth Rock Assurance** (Unknown): Send a polite follow-up email to the re